In [1]:
import pandas as pd
import nltk as nltk
import sklearn
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer,SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
import numpy as np
import html
import re

from IPython.display import clear_output
import math


Using TensorFlow backend.


In [2]:
pd.set_option(
    'display.max_colwidth', 0
)
np.random.seed(42)

In [3]:
nltk.download('subjectivity')


[nltk_data] Downloading package subjectivity to
[nltk_data]     /Users/ekaterinaromanovskaya/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!


True

In [4]:
df = pd.read_csv('hacker_news_sample.csv',nrows=1000)


In [5]:
def preprocess_df(df):
    df = df[df['text'].notna()]
    df['neg'], df['neu'], df['pos'], df["compound"], df["subjectivity"] = [np.nan, np.nan,np.nan,np.nan,np.nan]    
    return df

def add_sentiment_subj(df):
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["text"]
        text = html.unescape(text)
        text =  re.sub('<[^<]+?>', '', text)
        df.at[i,"text"] = text
        #subj = nltk.sentiment.util.demo_sent_subjectivity(text)
        #df.at[i,"subjectivity"] = subj
        ss = sid.polarity_scores(text)
        for k in ss:
            df.at[i,k] = ss[k]
        
pipeline = Pipeline(
    [
        ('vect', CountVectorizer(analyzer='word')),
        ('tfidf', TfidfTransformer()),    
    ]
)   



In [5]:
df = preprocess_df(df)
add_sentiment_subj(df)
#df.describe()

In [10]:
X_train, X_test = train_test_split(df)


In [11]:
scored = X_train.loc[X_train['compound'] != 0]

In [13]:
X = pipeline.fit_transform(scored["text"])

## Thread Reconstruction

In [5]:
import anytree
from anytree import *
from anytree.search import find
from anytree import Node, RenderTree, AsciiStyle
import json
from anytree import AnyNode
from anytree.exporter import JsonExporter
from anytree.exporter import DotExporter
from anytree.exporter import DictExporter

def checkpar(x, lst):
    if (x.parent is not None):
        return x.parent.name in lst
    else:
        None

nodes = []
for i, row in df.iterrows():    
    clear_output(wait=True)
    print(i)
    foundNode = None
    thisNode = None
    myid = str(int(row['id']))
    searcharr = [myid]
    parent = None
    appended = False
    if not math.isnan(row["parent"]):
        parent = str(int(row["parent"]))
    if len(nodes) > 0:        
        for oldnode in nodes:
            #our guy
            foundNodes = findall(oldnode, filter_=lambda nd: checkpar(nd, searcharr))
            if len(foundNodes) > 0:
                if appended == False:
                    if parent is not None:
                        thisNode = Node(myid,parent=Node(parent))
                    else:
                        thisNode = Node(myid)
                    nodes.append(thisNode)
                    appended = True
                for node in foundNodes:
                        node.parent = thisNode                                    
    if appended == False:
        if parent is not None:
            thisNode = Node(myid,parent=Node(parent))
        else:
            thisNode = Node(myid)
        nodes.append(thisNode)



exporter = JsonExporter(indent=2)
for nd in nodes:
    if nd.height > 0:
        js  = exporter.export(nd)
        with open(f"./jsons/{root.name}.json", 'w') as outfile:
            json.dump(js, outfile)
        DotExporter(nd).to_picture(f"./jsons/pngs/{nd.name}.png")            


1351


KeyboardInterrupt: 

In [12]:
nodes[2].height

0

In [29]:
from anytree.exporter import JsonExporter
from anytree.exporter import DotExporter

import json
root = Node(name="root")
s0 = Node(name="sub0", parent=root)
s0a = Node(name="sub0A", parent=s0)
s0b = Node(name="sub0B", parent=s0)
s1 = Node(name="sub1", parent=root)
exporter = JsonExporter(indent=2, sort_keys=True)
js  = exporter.export(root)
print(js)
with open(f"./jsons/{root.name}.json", 'w') as outfile:
    json.dump(js, outfile)
DotExporter(root).to_picture(f"{root.name}.png")            


{
  "children": [
    {
      "children": [
        {
          "name": "sub0A"
        },
        {
          "name": "sub0B"
        }
      ],
      "name": "sub0"
    },
    {
      "name": "sub1"
    }
  ],
  "name": "root"
}


In [5]:
from collections import defaultdict

ROOT = -1

def create_tree(pairs):
    """ Given an iterable of (parent, child) identifiers, build a tree
        where each node is a dict whose key is its identifier, and whose
        value is a list of children (which may be empty for leaf nodes).
        Multiple root nodes are supported; all roots are placed under a
        synthetic ROOT node.
    """
    def node(): return [ROOT,[]]
    table = defaultdict(node)
    # Build 2-way mapping between nodes
    idx = 0
    for parent, child in pairs:
        idx = idx+1
        clear_output(wait=True)
        print(idx)
        table[parent][1].append(child)  # parent - > children
        table[child][0] = parent  # child -> parent

    def follow(parent, childids):
        for c in childids:
            empty = []
            child = {c: empty}
            parent.append(child)
            if c in table:
                follow(empty, table[c][1])
                
    # Recursively fill in the tree
    tree = {ROOT:[]}
    roots = [k for k,v in table.items() if v[0] == ROOT]
    follow(tree[ROOT], roots)   
    return tree

#pairs = [(1,2),(2,3),(2,4),(1,5),(6,7),(6,8),(8,9)]


In [7]:
import math
pairs = []
for i, row in df.iterrows():  
    myid = str(int(row['id']))
    parent = None
    if not math.isnan(row["parent"]):
        parent = str(int(row["parent"]))
    pair = (myid, parent)
    pairs.append(pair)

In [8]:
create_tree(pairs)

NameError: name 'clear_output' is not defined

## Counting from DF / users

In [47]:
for node in nodes:
    print(RenderTree(node,style=AsciiStyle()))

# node = nodes[0]
# node = Node('w')

# for w in ['w',"2","d"]:
#     Node(w,parent=node)


# print(RenderTree(node,style=AsciiStyle()))

Node('/7127578/7131680')
Node('/3145330/3146879')
Node('/11189361/11190089')
Node('/11189361/11190089')
Node('/8170071/8170491')
Node('/8170071/8170491')
Node('/8170071/8170491')
Node('/8170071/8170491')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/5327590/5330773')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/2452073/2454827')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232')
Node('/None/7425232'

In [28]:
print(RenderTree(node,style=AsciiStyle()))

Node('/7127578.0/7131680')


## Decision Tree

In [ ]:
pipeline = Pipeline(
    [
        ('vect', CountVectorizer(analyzer='word')),
        ('tfidf', TfidfTransformer()),    
    ]
)

In [122]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X, scored["compound"])

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [129]:
scored_test = X_test.loc[X_test['compound'] != 0]
X2 = pipeline.transform(scored_test["text"])
print(scored_test["compound"].shape)
print(X2.shape)

In [133]:
from sklearn.metrics import mean_squared_error
predictions = tree_reg.predict(X2)

tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.6365505601380854

## Keras 

In [14]:

EPOCHS = 10

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [15]:
model = build_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                406784    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 411,009
Trainable params: 411,009
Non-trainable params: 0
_________________________________________________________________


In [22]:
import joblib
joblib.dump(pipeline,'sklearn_pipeline.pkl')

['sklearn_pipeline.pkl']

In [23]:
pipeline = joblib.load('sklearn_pipeline.pkl')

In [16]:
filepath = "keras/model.hdf5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor="loss",
    verbose=1,
    mode="min",
    save_best_only=True)
history = model.fit(
  X, scored["compound"],
  epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=[checkpoint_callback])

Train on 405 samples, validate on 102 samples
Epoch 1/10
224/405 [===============>..............] - ETA: 0s - loss: 0.3283 - mae: 0.5106 - mse: 0.3283
Epoch 00001: loss improved from inf to 0.33121, saving model to keras/model.hdf5
405/405 [==============================] - 1s 2ms/sample - loss: 0.3312 - mae: 0.4999 - mse: 0.3312 - val_loss: 0.3451 - val_mae: 0.5079 - val_mse: 0.3451
Epoch 2/10
320/405 [======================>.......] - ETA: 0s - loss: 0.2432 - mae: 0.4019 - mse: 0.2432
Epoch 00002: loss improved from 0.33121 to 0.24344, saving model to keras/model.hdf5
405/405 [==============================] - 0s 293us/sample - loss: 0.2434 - mae: 0.4056 - mse: 0.2434 - val_loss: 0.3420 - val_mae: 0.5044 - val_mse: 0.3420
Epoch 3/10
352/405 [=========================>....] - ETA: 0s - loss: 0.1431 - mae: 0.2924 - mse: 0.1431
Epoch 00003: loss improved from 0.24344 to 0.14154, saving model to keras/model.hdf5
405/405 [==============================] - 0s 238us/sample - loss: 0.1415 - 

In [9]:
!pip install q keras==2.2.4
!pip install q scikit-learn==0.22.1
!pip install q tensorflow==2.0.0

     |████████████████████████████████| 312 kB 1.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
  Using cached scikit_learn-0.22.1-cp37-cp37m-macosx_10_6_intel.whl (11.0 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1
     |████████████████████████████████| 102.7 MB 255 kB/s 
     |████████████████████████████████| 3.8 MB 3.8 MB/s 
     |████████████████████████████████| 449 kB 13.4 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-

In [10]:
## LOADING EXTERNAL
model = keras.models.load_model("model.hdf5")
pipeline = joblib.load('sklearn_pipeline.pkl')
def predict_text(text, model = model):
    xtemp = pipeline.transform([text])
    predictions = model.predict(xtemp)
    print(predictions)
    return predictions

print(keras.__version__)
print(sklearn.__version__)
print(tf.__version__)

preds = predict_text("I hate all this all of you")


OSError: SavedModel file does not exist at: model.hdf5/{saved_model.pbtxt|saved_model.pb}

In [19]:
model = keras.models.load_model("./keras/model.hdf5")
# model = build_model()
# model.load_weights("./keras/")


In [160]:
predictions = model.predict(X2)


In [161]:
tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.39776707368686554

[nltk_data] Downloading package subjectivity to
[nltk_data]     /Users/stepanboltalin/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.
Cannot find the sentiment analyzer you want to load.
Training a new one using NaiveBayesClassifier.
Training classifier
Most Informative Features
            contains(--) = True             subj : obj    =     61.0 : 1.0
        contains(film's) = True             subj : obj    =     33.7 : 1.0
       contains(decides) = True              obj : subj   =     28.3 : 1.0
      contains(discover) = True              obj : subj   =     26.3 : 1.0
    contains(girlfriend) = True              obj : subj   =     25.0 : 1.0
          contains(town) = True              obj : subj   =     23.4 : 1.0
  contains(entertaining) = True             subj : obj    =     22.2 : 1.0
     contains(detective) = True              obj : subj   =     21.0 : 1.0
        contains(mother) = True              obj : subj   =     20.1 : 1.0
         contains(finds) = Tru

subj
